Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [58]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from tqdm import tqdm
import time

In [3]:
pickle_file = './input/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [41]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    #print("Initial shape", data.get_shape().as_list())
    
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    #print("After first conv stride (2,2) same padding",conv.get_shape().as_list())
    hidden = tf.nn.relu(conv + layer1_biases)

    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    #print("After second conv stride (2,2) same padding",conv.get_shape().as_list())
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()

    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    #print("After reshape",reshape.get_shape().as_list())

    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    #print("After hidden dense layer",hidden.get_shape().as_list())

    out = tf.matmul(hidden, layer4_weights) + layer4_biases
    #print("Output shape",out.get_shape().as_list())
    return out
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.706900
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.267071
Minibatch accuracy: 56.2%
Validation accuracy: 70.5%
Minibatch loss at step 200: 0.457166
Minibatch accuracy: 87.5%
Validation accuracy: 74.3%
Minibatch loss at step 300: 0.972690
Minibatch accuracy: 68.8%
Validation accuracy: 78.7%
Minibatch loss at step 400: 0.716878
Minibatch accuracy: 75.0%
Validation accuracy: 79.3%
Minibatch loss at step 500: 0.524336
Minibatch accuracy: 75.0%
Validation accuracy: 80.2%
Minibatch loss at step 600: 0.725250
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 700: 0.518035
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 800: 1.184052
Minibatch accuracy: 81.2%
Validation accuracy: 82.2%
Minibatch loss at step 900: 0.093484
Minibatch accuracy: 100.0%
Validation accuracy: 82.8%
Minibatch loss at step 1000: 0.574827
Minibatch accuracy: 87.5%
Validation accuracy: 82.3%

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [57]:
batch_size = 16 # Batch size.
img_size = 28 # Size of image.
n_channels = 1 # Number of channels in image.
num_hidden = 64 # Number of hidden nodes in the dense layer.
depth = 16 # Number of feature maps/depth of convolutions
num_labels = 10 # Number of output labels.
s = 2 # Stride-number used to reduce dimensionality.
stride_conv = [1, 1, 1, 1] # Stride for convolutionl layers.
stride_pool = [1, 2, 2, 1] # Stride for the pooling layers.
kernel_pool = [1, 2, 2, 1] # Kernel for the pooling layers.
patch_size = 5 # Patch size used for the convolutional layers.


graph = tf.Graph() # Init graph.

with graph.as_default():
    
    # Data.
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, img_size, img_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Weigths and biases for the first convolutional layer.
    l1_w = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    l1_b = tf.Variable(tf.zeros([depth]))
    
    # Weigths and biases for the first convolutional layer.
    l2_w = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    l2_b = tf.Variable(tf.constant(1.0, shape=[depth])) # Constant biases which will not change during training

    # Weigths and biases for the hidden dense layer.
    l3_w = tf.Variable(tf.truncated_normal([img_size // s**2 * img_size // s**2 * depth, num_hidden], stddev=0.1))
    l3_b = tf.Variable(tf.constant(1.0, shape=[num_hidden])) # Constant biases which will not change during training
    
    # Weigths and biases for the last dense output layer.
    l4_w = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    l4_b = tf.Variable(tf.constant(1.0, shape=[num_labels])) # Constant biases which will not change during training
    
    # Define our model
    def model(data):
        # First convolutional layer, relu activation.
        conv1 = tf.nn.conv2d(data, l1_w, stride_conv, padding = 'SAME')
        out_conv1 = tf.nn.relu(conv1 + l1_b)
        
        # First pooling layer. 
        pool1 = tf.nn.max_pool(out_conv1, kernel_pool, stride_pool, padding = 'SAME')
        
        # Second convolutional layer, relu activation.
        conv2 = tf.nn.conv2d(pool1, l2_w, stride_conv, padding = 'SAME')
        out_conv2 = tf.nn.relu(conv2 + l2_b)
        
        # Second pooling layer. 
        pool2 = tf.nn.max_pool(out_conv2, kernel_pool, stride_pool, padding = 'SAME')
        
        # Get shape of output from second pooling layer.
        shape = pool2.get_shape().as_list()
        
        # Reshaping layer to flatten it.
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        
        # Hidden layer.
        hidden = tf.nn.relu(tf.matmul(reshape, l3_w) + l3_b)
        
        # Output
        out = tf.matmul(hidden, l4_w) + l4_b
        return out

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [59]:
num_epoch=10 # Number of epochs.
num_steps=500 # Number of steps in each epoch.

with tf.Session(graph=graph) as session:
    print("Number of epochs: ", num_epoch) 
    print("Steps in each epoch: ", num_steps)
    tf.global_variables_initializer().run() # Ensures the parameters get initialized as specified.
    print("Initialized")
    time.sleep(1.5) # Sleep to prevent progressbar to "leak"
    # Iterate over epochs.
    for i in range(num_epoch+1):
        print("="*45 + " Epoch: " + str(i) + " " + "="*45)
        time.sleep(1.5)
        for step in tqdm(range(num_steps), total=num_steps):
            permutation = np.random.randint(0,train_labels.shape[0], batch_size)
            batch_data = train_dataset[permutation,:]
            batch_labels = train_labels[permutation]

            # Define dictionary telling the session where to feed the minibatch.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

            # Run the operations on this graph 
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)

        print("Minibatch loss: %f" % (l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        time.sleep(1)

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Number of epochs:  10
Steps in each epoch:  500
Initialized
============================================= Epoch: 0 =============================================


100%|██████████| 500/500 [00:07<00:00, 64.10it/s]


Minibatch loss: 0.754470
Minibatch accuracy: 75.0%
Validation accuracy: 80.3%
============================================= Epoch: 1 =============================================


100%|██████████| 500/500 [00:08<00:00, 56.05it/s]


Minibatch loss: 0.552077
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%
============================================= Epoch: 2 =============================================


...

Minibatch loss: 0.344032
Minibatch accuracy: 93.8%
Validation accuracy: 88.2%
============================================= Epoch: 9 =============================================


100%|██████████| 500/500 [00:08<00:00, 61.42it/s]


Minibatch loss: 0.266269
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
============================================= Epoch: 10 =============================================


100%|██████████| 500/500 [00:08<00:00, 59.80it/s]


Minibatch loss: 0.398274
Minibatch accuracy: 81.2%
Validation accuracy: 88.5%
Test accuracy: 94.0%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

The previous model did not seem to overfit, so lets try adding some complexity, dropout and learning rate decay.

In [84]:
batch_size = 16 # Batch size.
img_size = 28 # Size of image.
n_channels = 1 # Number of channels in image.
num_hidden = 104 # Number of hidden nodes in the dense layer.
depth = 18 # Number of feature maps/depth of convolutions
num_labels = 10 # Number of output labels.
s = 2 # Stride-number used to reduce dimensionality.
stride_conv = [1, 1, 1, 1] # Stride for convolutionl layers.
stride_pool = [1, 2, 2, 1] # Stride for the pooling layers.
kernel_pool = [1, 2, 2, 1] # Kernel for the pooling layers.
patch_size = 5 # Patch size used for the convolutional layers.
dropout_rate = 0.5 # Dropout rate.


graph = tf.Graph() # Init graph.

with graph.as_default():
    
    # Data.
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, img_size, img_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Weigths and biases for the first convolutional layer.
    l1_w = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    l1_b = tf.Variable(tf.zeros([depth]))
    
    # Weigths and biases for the first convolutional layer.
    l2_w = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    l2_b = tf.Variable(tf.constant(1.0, shape=[depth])) # Constant biases which will not change during training

    # Weigths and biases for the hidden dense layer.
    l3_w = tf.Variable(tf.truncated_normal([img_size // s**2 * img_size // s**2 * depth, num_hidden], stddev=0.1))
    l3_b = tf.Variable(tf.constant(1.0, shape=[num_hidden])) # Constant biases which will not change during training
    
    # Weigths and biases for the last dense output layer.
    l4_w = tf.Variable(tf.truncated_normal([num_hidden, num_hidden], stddev=0.1))
    l4_b = tf.Variable(tf.constant(1.0, shape=[num_hidden])) # Constant biases which will not change during training
    
    # Weigths and biases for the last dense output layer.
    l5_w = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    l5_b = tf.Variable(tf.constant(1.0, shape=[num_labels])) # Constant biases which will not change during training
    
    # Define our model
    def model(data):
        # First convolutional layer, relu activation.
        conv1 = tf.nn.conv2d(data, l1_w, stride_conv, padding = 'SAME')
        out_conv1 = tf.nn.relu(conv1 + l1_b)
        
        # First pooling layer. 
        pool1 = tf.nn.max_pool(out_conv1, kernel_pool, stride_pool, padding = 'SAME')
        
        # Second convolutional layer, relu activation.
        conv2 = tf.nn.conv2d(pool1, l2_w, stride_conv, padding = 'SAME')
        out_conv2 = tf.nn.relu(conv2 + l2_b)
        
        # Second pooling layer. 
        pool2 = tf.nn.max_pool(out_conv2, kernel_pool, stride_pool, padding = 'SAME')
        
        # Get shape of output from second pooling layer.
        shape = pool2.get_shape().as_list()
        
        # Reshaping layer to flatten it.
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        
        # First Hidden layer.
        hidden1 = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, l3_w) + l3_b), dropout_rate)
        
        # Second Hidden layer.
        hidden2 = tf.nn.dropout(tf.matmul(hidden1, l4_w) + l4_b, dropout_rate)
        
        # Output.
        out = tf.matmul(hidden2, l5_w) + l5_b

        return out

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    #optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.08, global_step, 2000, 0.9) # Learning rate decay:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [85]:
num_epoch=30 # Number of epochs.
num_steps=2000 # Number of steps in each epoch.

with tf.Session(graph=graph) as session:
    print("Number of epochs: ", num_epoch) 
    print("Steps in each epoch: ", num_steps)
    tf.global_variables_initializer().run() # Ensures the parameters get initialized as specified.
    print("Initialized")
    time.sleep(1.5) # Sleep to prevent progressbar to "leak"
    # Iterate over epochs.
    for i in range(num_epoch+1):
        print("="*45 + " Epoch: " + str(i) + " " + "="*45)
        time.sleep(1.5)
        for step in tqdm(range(num_steps), total=num_steps):
            permutation = np.random.randint(0,train_labels.shape[0], batch_size)
            batch_data = train_dataset[permutation,:]
            batch_labels = train_labels[permutation]

            # Define dictionary telling the session where to feed the minibatch.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}

            # Run the operations on this graph 
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)

        print("Minibatch loss: %f" % (l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        time.sleep(1)

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Number of epochs:  30
Steps in each epoch:  2000
Initialized
============================================= Epoch: 0 =============================================


100%|██████████| 2000/2000 [00:41<00:00, 47.87it/s]


Minibatch loss: 0.825007
Minibatch accuracy: 75.0%
Validation accuracy: 80.4%
============================================= Epoch: 1 =============================================


100%|██████████| 2000/2000 [00:38<00:00, 52.20it/s]


Minibatch loss: 0.043933
Minibatch accuracy: 100.0%
Validation accuracy: 83.1%
============================================= Epoch: 2 =============================================


...

Minibatch loss: 0.157371
Minibatch accuracy: 93.8%
Validation accuracy: 88.9%
============================================= Epoch: 29 =============================================


100%|██████████| 2000/2000 [00:30<00:00, 65.02it/s]


Minibatch loss: 0.274732
Minibatch accuracy: 87.5%
Validation accuracy: 89.3%
============================================= Epoch: 30 =============================================


100%|██████████| 2000/2000 [00:30<00:00, 65.02it/s]


Minibatch loss: 0.405457
Minibatch accuracy: 93.8%
Validation accuracy: 89.4%
Test accuracy: 94.6%


Good enough.